In [10]:
# Install required packages
!pip install pandas numpy scikit-learn matplotlib seaborn joblib nltk


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [11]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import joblib
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


[nltk_data] Downloading package punkt to /home/basar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/basar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 1. Load Processed Datasets

In [12]:
# Load all processed datasets
print("📂 Loading datasets...\n")

try:
    df_nutrition = pd.read_csv('../datasets/processed/nutrition_processed.csv')
    print(f"✓ Nutrition: {len(df_nutrition)} records")
except:
    print("⚠ Nutrition dataset not found")
    df_nutrition = pd.DataFrame()

try:
    df_exercise = pd.read_csv('../datasets/processed/exercise_processed.csv')
    print(f"✓ Exercise: {len(df_exercise)} records")
except:
    print("⚠ Exercise dataset not found")
    df_exercise = pd.DataFrame()

try:
    df_qa = pd.read_csv('../datasets/processed/medical_qa_processed.csv')
    print(f"✓ Medical Q&A: {len(df_qa)} records")
except:
    print("⚠ Medical Q&A dataset not found")
    df_qa = pd.DataFrame()

try:
    df_pregnancy = pd.read_csv('../datasets/processed/pregnancy_processed.csv')
    print(f"✓ Pregnancy: {len(df_pregnancy)} records")
except:
    print("⚠ Pregnancy dataset not found")
    df_pregnancy = pd.DataFrame()

try:
    df_womens = pd.read_csv('../datasets/processed/womens_health_processed.csv')
    print(f"✓ Women's Health: {len(df_womens)} records")
except:
    print("⚠ Women's Health dataset not found")
    df_womens = pd.DataFrame()

try:
    knowledge_base = pd.read_csv('../datasets/processed/knowledge_base.csv')
    print(f"✓ Knowledge Base: {len(knowledge_base)} entries")
except:
    print("⚠ Knowledge Base not found")
    knowledge_base = pd.DataFrame()

print("\n✅ Datasets loaded successfully!")

📂 Loading datasets...

⚠ Nutrition dataset not found
⚠ Exercise dataset not found
⚠ Medical Q&A dataset not found
⚠ Pregnancy dataset not found
⚠ Women's Health dataset not found
⚠ Knowledge Base not found

✅ Datasets loaded successfully!
⚠ Women's Health dataset not found
⚠ Knowledge Base not found

✅ Datasets loaded successfully!


## 2. Train Q&A Model (TF-IDF + Cosine Similarity)

In [13]:
print("🤖 Training Q&A Model...\n")

if not knowledge_base.empty:
    # Prepare Q&A data
    questions = knowledge_base['question'].fillna('').tolist()
    answers = knowledge_base['answer'].fillna('').tolist()
    categories = knowledge_base['category'].fillna('general').tolist()
    
    # Create TF-IDF vectorizer
    vectorizer = TfidfVectorizer(
        max_features=500,
        stop_words='english',
        ngram_range=(1, 2),
        min_df=1
    )
    
    # Fit on questions
    question_vectors = vectorizer.fit_transform(questions)
    
    print(f"✓ Vectorizer trained on {len(questions)} questions")
    print(f"✓ Vocabulary size: {len(vectorizer.vocabulary_)}")
    
    # Test the model
    test_questions = [
        "How to improve health?",
        "What is healthy blood sugar?",
        "Best foods for heart?"
    ]
    
    print("\n🧪 Testing Q&A Model:\n")
    for test_q in test_questions:
        test_vec = vectorizer.transform([test_q])
        similarities = cosine_similarity(test_vec, question_vectors)[0]
        best_match_idx = similarities.argmax()
        
        if similarities[best_match_idx] > 0.1:
            print(f"Q: {test_q}")
            print(f"A: {answers[best_match_idx][:100]}...")
            print(f"Similarity: {similarities[best_match_idx]:.2f}\n")
    
    # Save model
    joblib.dump(vectorizer, '../models/qa_vectorizer.pkl')
    
    # Save Q&A database
    qa_db = {
        'questions': questions,
        'answers': answers,
        'categories': categories
    }
    joblib.dump(qa_db, '../models/qa_database.pkl')
    
    print("✅ Q&A Model saved successfully!")
else:
    print("❌ No knowledge base available for training")

🤖 Training Q&A Model...

❌ No knowledge base available for training


## 3. Train Calorie Prediction Model

In [14]:
print("🍽 Training Calorie Prediction Model...\n")

if not df_nutrition.empty and 'protein' in df_nutrition.columns:
    # Prepare features
    X = df_nutrition[['protein', 'carbs', 'fat']]
    y = df_nutrition['calories']
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Train model
    calorie_model = RandomForestRegressor(n_estimators=100, random_state=42)
    calorie_model.fit(X_train, y_train)
    
    # Evaluate
    y_pred = calorie_model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    print(f"✓ Model trained on {len(X_train)} samples")
    print(f"✓ R² Score: {r2:.4f}")
    print(f"✓ RMSE: {rmse:.2f} calories\n")
    
    # Test predictions
    print("🧪 Test Predictions:\n")
    for i in range(min(3, len(X_test))):
        print(f"Protein: {X_test.iloc[i]['protein']}g, Carbs: {X_test.iloc[i]['carbs']}g, Fat: {X_test.iloc[i]['fat']}g")
        print(f"Predicted: {y_pred[i]:.0f} cal, Actual: {y_test.iloc[i]:.0f} cal\n")
    
    # Save model
    joblib.dump(calorie_model, '../models/calorie_predictor.pkl')
    print("✅ Calorie Prediction Model saved!")
else:
    print("❌ Insufficient nutrition data for training")

🍽 Training Calorie Prediction Model...

❌ Insufficient nutrition data for training


## 4. Train Exercise Recommendation Model

In [15]:
print("🏃 Training Exercise Recommendation Model...\n")

if not df_exercise.empty and 'body_weight' in df_exercise.columns:
    # Prepare features
    X = df_exercise[['body_weight', 'calories_per_hour']]
    
    # Encode exercise names
    le_exercise = LabelEncoder()
    y = le_exercise.fit_transform(df_exercise['exercise_name'])
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Train model
    exercise_model = RandomForestClassifier(n_estimators=100, random_state=42)
    exercise_model.fit(X_train, y_train)
    
    # Evaluate
    y_pred = exercise_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"✓ Model trained on {len(X_train)} samples")
    print(f"✓ Accuracy: {accuracy:.4f}\n")
    
    # Save model
    joblib.dump(exercise_model, '../models/exercise_recommender.pkl')
    joblib.dump(le_exercise, '../models/exercise_encoder.pkl')
    print("✅ Exercise Recommendation Model saved!")
else:
    print("❌ Insufficient exercise data for training")

🏃 Training Exercise Recommendation Model...

❌ Insufficient exercise data for training


## 5. Create Health Advisor Knowledge Base

In [16]:
print("🏥 Creating Health Advisor Knowledge Base...\n")

# Combine all health knowledge
health_knowledge = {
    'bmi_ranges': {
        'underweight': {'range': (0, 18.5), 'advice': 'Increase calorie intake with nutritious foods. Consult a nutritionist.'},
        'normal': {'range': (18.5, 24.9), 'advice': 'Maintain current healthy lifestyle and balanced diet.'},
        'overweight': {'range': (25, 29.9), 'advice': 'Reduce calorie intake, increase physical activity. Consider portion control.'},
        'obese': {'range': (30, 100), 'advice': 'Consult healthcare provider. Focus on gradual weight loss through diet and exercise.'}
    },
    'blood_pressure': {
        'normal': {'range': {'systolic': (90, 120), 'diastolic': (60, 80)}, 'advice': 'Maintain healthy lifestyle'},
        'elevated': {'range': {'systolic': (120, 129), 'diastolic': (60, 80)}, 'advice': 'Reduce salt, exercise regularly'},
        'high': {'range': {'systolic': (130, 200), 'diastolic': (80, 120)}, 'advice': 'Consult doctor, medication may be needed'}
    },
    'blood_glucose': {
        'normal': {'range': (70, 100), 'advice': 'Maintain balanced diet and regular exercise'},
        'prediabetes': {'range': (100, 125), 'advice': 'Reduce sugar intake, increase physical activity, monitor regularly'},
        'diabetes': {'range': (126, 500), 'advice': 'Consult doctor immediately, medication and diet control essential'}
    },
    'daily_water': {
        'formula': 'body_weight_kg * 0.033',  # liters
        'min': 2.0,
        'max': 4.0
    },
    'daily_steps': {
        'sedentary': 5000,
        'moderate': 7500,
        'active': 10000,
        'very_active': 12500
    },
    'sleep_hours': {
        'adult': (7, 9),
        'teenager': (8, 10),
        'child': (9, 12)
    }
}

# Add pregnancy knowledge
if not df_pregnancy.empty:
    pregnancy_dict = df_pregnancy.set_index('week').to_dict('index')
    health_knowledge['pregnancy'] = pregnancy_dict

# Add women's health knowledge
if not df_womens.empty:
    womens_health_dict = df_womens.set_index('symptom').to_dict('index')
    health_knowledge['womens_health'] = womens_health_dict

# Save knowledge base
with open('../models/health_knowledge.json', 'w') as f:
    json.dump(health_knowledge, f, indent=2)

print("✅ Health Knowledge Base created!")
print(f"\n📊 Knowledge Base contains:")
print(f"  - BMI Guidelines: {len(health_knowledge['bmi_ranges'])} categories")
print(f"  - Blood Pressure: {len(health_knowledge['blood_pressure'])} categories")
print(f"  - Blood Glucose: {len(health_knowledge['blood_glucose'])} categories")
print(f"  - Daily Water Calculator")
print(f"  - Step Goals: {len(health_knowledge['daily_steps'])} levels")
print(f"  - Sleep Recommendations")
if 'pregnancy' in health_knowledge:
    print(f"  - Pregnancy Guidance: {len(health_knowledge['pregnancy'])} weeks")
if 'womens_health' in health_knowledge:
    print(f"  - Women's Health: {len(health_knowledge['womens_health'])} symptoms")

🏥 Creating Health Advisor Knowledge Base...

✅ Health Knowledge Base created!

📊 Knowledge Base contains:
  - BMI Guidelines: 4 categories
  - Blood Pressure: 3 categories
  - Blood Glucose: 3 categories
  - Daily Water Calculator
  - Step Goals: 4 levels
  - Sleep Recommendations


## 6. Model Summary & Evaluation

In [17]:
print("="*60)
print("✅ HealthNest AI Model Training Complete!")
print("="*60)

print("\n📦 Trained Models:")
print("  1. Q&A Model (TF-IDF + Cosine Similarity)")
print("  2. Calorie Prediction Model (Random Forest Regressor)")
print("  3. Exercise Recommendation Model (Random Forest Classifier)")
print("  4. Health Knowledge Base (Rule-based System)")

print("\n💾 Saved Files:")
print("  - ../models/qa_vectorizer.pkl")
print("  - ../models/qa_database.pkl")
print("  - ../models/calorie_predictor.pkl")
print("  - ../models/exercise_recommender.pkl")
print("  - ../models/exercise_encoder.pkl")
print("  - ../models/health_knowledge.json")

print("\n📝 Next Steps:")
print("  1. Test models using test_model.py")
print("  2. Deploy backend API using Flask")
print("  3. Build frontend chatbot UI")
print("  4. Integrate with HealthNest app")

✅ HealthNest AI Model Training Complete!

📦 Trained Models:
  1. Q&A Model (TF-IDF + Cosine Similarity)
  2. Calorie Prediction Model (Random Forest Regressor)
  3. Exercise Recommendation Model (Random Forest Classifier)
  4. Health Knowledge Base (Rule-based System)

💾 Saved Files:
  - ../models/qa_vectorizer.pkl
  - ../models/qa_database.pkl
  - ../models/calorie_predictor.pkl
  - ../models/exercise_recommender.pkl
  - ../models/exercise_encoder.pkl
  - ../models/health_knowledge.json

📝 Next Steps:
  1. Test models using test_model.py
  2. Deploy backend API using Flask
  3. Build frontend chatbot UI
  4. Integrate with HealthNest app


## 7. Save Model Metadata

In [18]:
# Save model metadata
metadata = {
    'project': 'HealthNest AI Health Assistant',
    'version': '1.0.0',
    'date_trained': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
    'models': {
        'qa_model': {
            'type': 'TF-IDF + Cosine Similarity',
            'knowledge_base_size': len(knowledge_base) if not knowledge_base.empty else 0,
            'vocabulary_size': len(vectorizer.vocabulary_) if 'vectorizer' in locals() else 0
        },
        'calorie_predictor': {
            'type': 'Random Forest Regressor',
            'training_samples': len(X_train) if 'X_train' in locals() else 0,
            'r2_score': r2 if 'r2' in locals() else None
        },
        'exercise_recommender': {
            'type': 'Random Forest Classifier',
            'accuracy': accuracy if 'accuracy' in locals() else None
        }
    },
    'datasets': {
        'nutrition': len(df_nutrition) if not df_nutrition.empty else 0,
        'exercise': len(df_exercise) if not df_exercise.empty else 0,
        'medical_qa': len(df_qa) if not df_qa.empty else 0,
        'pregnancy': len(df_pregnancy) if not df_pregnancy.empty else 0,
        'womens_health': len(df_womens) if not df_womens.empty else 0
    }
}

with open('../models/model_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print("✅ Model metadata saved!")
print("\n📄 View metadata: ../models/model_metadata.json")

✅ Model metadata saved!

📄 View metadata: ../models/model_metadata.json
